In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import *
import keras.backend as K
from keras.layers import *
from keras.utils import *
from keras.regularizers import *
import numpy as np
import _pickle as pickle

W0504 00:10:49.963731 10084 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [2]:
embed = hub.Module('./embeddings/GUSE')

In [3]:
def GUSE(param):
    return embed(tf.squeeze(tf.cast(param, tf.string)), signature="default", as_dict=True)["default"]

In [4]:
input_layer = Input(shape=(1,), dtype="string")
guse = Lambda(GUSE, output_shape=(512,))(input_layer)
dense1 = Dense(256, activation="relu", kernel_regularizer=l2(0.005))(guse)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout1)
dropout2 = Dropout(0.5)(dense2)
dense3 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout2)
dropout3 = Dropout(0.5)(dense3)
dense4 = Dense(64, activation="relu", kernel_regularizer=l2(0.005))(dropout3)
dropout4 = Dropout(0.5)(dense4)
output = Dense(1, activation="sigmoid")(dropout4)
model = Model(inputs=[input_layer], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0504 00:10:51.703107 10084 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
__________

In [5]:
twitter_f = open("twitter_irony_all.pickle", mode="rb")
twitter_data = pickle.load(twitter_f)
a = np.zeros((156,1))
b = np.ones((156,1))
twitter_y = b
twitter_x = np.asarray(twitter_data)
twitter_x = np.reshape(twitter_x, (twitter_x.shape[0],1))

In [6]:
session = tf.Session()
K.set_session(session)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
model.load_weights("guse_model2.h5")
scores = model.evaluate(twitter_x, twitter_y)
scores[1]

156/156 [==============================] - 1s 7ms/step


0.8717948717948718

In [7]:
preds = model.predict(twitter_x)
preds

array([[0.68539107],
       [0.83285445],
       [0.06659451],
       [0.5137295 ],
       [0.8215864 ],
       [0.81894183],
       [0.8223159 ],
       [0.82760954],
       [0.6895938 ],
       [0.14925408],
       [0.8283279 ],
       [0.8120966 ],
       [0.82675904],
       [0.8204971 ],
       [0.84651464],
       [0.7491567 ],
       [0.83354354],
       [0.72429264],
       [0.8288956 ],
       [0.8324604 ],
       [0.8373509 ],
       [0.8003315 ],
       [0.81941676],
       [0.8200142 ],
       [0.81877184],
       [0.55667233],
       [0.83385515],
       [0.8257985 ],
       [0.82588464],
       [0.7963106 ],
       [0.55460984],
       [0.82201535],
       [0.8252736 ],
       [0.79609656],
       [0.8240837 ],
       [0.76231396],
       [0.81472355],
       [0.83150834],
       [0.824762  ],
       [0.7002792 ],
       [0.8246488 ],
       [0.8341392 ],
       [0.85033643],
       [0.3040255 ],
       [0.8134942 ],
       [0.03060776],
       [0.82122594],
       [0.724

In [8]:
twitter_x[7]

array(['My financial state of affairs is depressing and I cannot afford therapy...  or '],
      dtype='<U137')

In [16]:
twitter_x[155]

array(["See this be this bullshit, love will make you save a dead person for someone that don't even care. "],
      dtype='<U137')

In [20]:
something = ["I hope you all enjoyed my presentation today", '']
smt = np.asarray(something)
smt = np.reshape(smt, (smt.shape[0],1))
pred = model.predict(smt)
pred

array([[0.29589927],
       [0.13199294]], dtype=float32)